## 各种import

In [1]:
!pip install 'keras<2'

    100% |████████████████████████████████| 184kB 3.3MB/s 
    100% |████████████████████████████████| 2.8MB 415kB/s 
  Running setup.py bdist_wheel for keras ... - done
  Stored in directory: /content/.cache/pip/wheels/f6/c5/63/97d96b41bf822858027c70b04448c19deaccf1cf518148fa82
  Running setup.py bdist_wheel for theano ... - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/46/a2/7d/b4cac381d5151daa9f9e0b3e4e4b65edaea6355ae296c97cf2
Successfully built keras theano


In [2]:
from __future__ import division,print_function
import math, os, json, sys, re
import cPickle as pickle
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
from operator import itemgetter, attrgetter, methodcaller
from collections import OrderedDict
import itertools
from itertools import chain

import pandas as pd
import PIL
from PIL import Image
from numpy.random import random, permutation, randn, normal, uniform, choice
from numpy import newaxis
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE

from IPython.lib.display import FileLink

import theano
from theano import shared, tensor as T
from theano.tensor.nnet import conv2d, nnet
from theano.tensor.signal import pool

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [3]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("../examples"))

import utils; reload(utils)
from utils import *

from __future__ import division, print_function

ImportError: ignored

## 从Keras导入数据

In [3]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## 处理数据

### One-Hot-Encoding

In [4]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道
对于tensorflow, 前两个维度代表图片大小, 第三个维度代表色彩通道, 所以np.expand_dims(trn_data, 3)
对于thenao, 第一个维度代表色彩通道, 后两个维度代表图片大小, 所以np.expand_dims(trn_data, 1)

In [5]:
trn_data = np.expand_dims(trn_data, 3)
test_data = np.expand_dims(test_data, 3)
trn_data.shape

(60000, 28, 28, 1)

### 对输入图像进行正则化

In [0]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

### 将输入数据转化为batches

In [0]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

## 1. 简单神经网络模型

In [0]:
simpleNN = Sequential()
simpleNN.add(Lambda(norm_input, input_shape=(28, 28, 1)))
simpleNN.add(Flatten())
simpleNN.add(Dense(512, activation='relu'))
simpleNN.add(Dense(10, activation='softmax'))
simpleNN.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

### 1.1 直接训练

In [18]:
#simpleNN.optimizer.lr = 0.001
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 15s - loss: 0.1884 - acc: 0.9417 - val_loss: 0.0966 - val_acc: 0.9697


In [19]:
simpleNN.optimizer.lr = 0.0001
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 15s - loss: 0.0852 - acc: 0.9734 - val_loss: 0.0963 - val_acc: 0.9707
Epoch 2/3
48160/60000 [=======================>......] - ETA: 2s - loss: 0.0625 - acc: 0.9802

60000/60000 [==============================] - 15s - loss: 0.0622 - acc: 0.9803 - val_loss: 0.0862 - val_acc: 0.9739
Epoch 3/3
60000/60000 [==============================] - 15s - loss: 0.0495 - acc: 0.9841 - val_loss: 0.0963 - val_acc: 0.9724


### 1.2 生成batch再训练

利用ImageDataGenerator(), 使用fit_generator而不是fit
注意fit_generator的文档中参数由于版本升级发生了变化

In [17]:
simpleNN.optimizer.lr = 0.001
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 13s - loss: 0.2080 - acc: 0.9389 - val_loss: 0.1421 - val_acc: 0.9528


In [18]:
simpleNN.optimizer.lr = 0.0001
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 14s - loss: 0.0846 - acc: 0.9747 - val_loss: 0.0835 - val_acc: 0.9744
Epoch 2/3
60000/60000 [==============================] - 14s - loss: 0.0560 - acc: 0.9838 - val_loss: 0.0734 - val_acc: 0.9777
Epoch 3/3
22464/60000 [==========>...................] - ETA: 8s - loss: 0.0395 - acc: 0.9898

60000/60000 [==============================] - 14s - loss: 0.0389 - acc: 0.9895 - val_loss: 0.0682 - val_acc: 0.9788


## 2. 类似vgg16的卷积神经网络模型

注意比较各类activation之间的关系, 只有最后一个线性层使用softmax使得结果One-Hot-Coded, 其余层使用relu

In [0]:
def getSimpleCNN():
  simpleCNN = Sequential([
      Lambda(norm_input, input_shape=(28, 28, 1)),
      Convolution2D(32, 3, 3, activation="relu"),
      Convolution2D(32, 3, 3, activation="relu"),
      MaxPooling2D(),
      Convolution2D(64, 3, 3, activation="relu"),
      Convolution2D(64, 3, 3, activation="relu"),
      MaxPooling2D(),
      Flatten(),
      Dense(512, activation="relu"),
      Dense(10, activation="softmax")])
  simpleCNN.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
  return simpleCNN

In [21]:
simpleCNN = getSimpleCNN()
simpleCNN.optimizer.lr=0.01
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 159s - loss: 0.3850 - acc: 0.8837 - val_loss: 0.1118 - val_acc: 0.9652
Epoch 2/3
 3264/60000 [>.............................] - ETA: 143s - loss: 0.1040 - acc: 0.9697

60000/60000 [==============================] - 159s - loss: 0.0931 - acc: 0.9718 - val_loss: 0.0640 - val_acc: 0.9801
Epoch 3/3
 7552/60000 [==>...........................] - ETA: 131s - loss: 0.0638 - acc: 0.9791

60000/60000 [==============================] - 157s - loss: 0.0651 - acc: 0.9802 - val_loss: 0.0534 - val_acc: 0.9818


In [22]:
simpleCNN.optimizer.lr=0.001
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/10
60000/60000 [==============================] - 159s - loss: 0.0508 - acc: 0.9839 - val_loss: 0.0493 - val_acc: 0.9830
Epoch 2/10
 3008/60000 [>.............................] - ETA: 143s - loss: 0.0399 - acc: 0.9877

60000/60000 [==============================] - 159s - loss: 0.0428 - acc: 0.9868 - val_loss: 0.0509 - val_acc: 0.9839
Epoch 3/10
 7424/60000 [==>...........................] - ETA: 133s - loss: 0.0412 - acc: 0.9875

60000/60000 [==============================] - 161s - loss: 0.0359 - acc: 0.9890 - val_loss: 0.0461 - val_acc: 0.9859
Epoch 4/10
 9152/60000 [===>..........................] - ETA: 130s - loss: 0.0239 - acc: 0.9920

60000/60000 [==============================] - 161s - loss: 0.0317 - acc: 0.9901 - val_loss: 0.0471 - val_acc: 0.9850
Epoch 5/10
 9856/60000 [===>..........................] - ETA: 128s - loss: 0.0267 - acc: 0.9922

60000/60000 [==============================] - 162s - loss: 0.0283 - acc: 0.9907 - val_loss: 0.0347 - val_acc: 0.9879
Epoch 6/10
10112/60000 [====>.........................] - ETA: 129s - loss: 0.0225 - acc: 0.9932

60000/60000 [==============================] - 164s - loss: 0.0237 - acc: 0.9928 - val_loss: 0.0365 - val_acc: 0.9885
Epoch 7/10
10240/60000 [====>.........................] - ETA: 129s - loss: 0.0184 - acc: 0.9943

60000/60000 [==============================] - 163s - loss: 0.0216 - acc: 0.9932 - val_loss: 0.0442 - val_acc: 0.9853
Epoch 8/10
10240/60000 [====>.........................] - ETA: 128s - loss: 0.0151 - acc: 0.9958

60000/60000 [==============================] - 162s - loss: 0.0179 - acc: 0.9943 - val_loss: 0.0378 - val_acc: 0.9887
Epoch 9/10
10240/60000 [====>.........................] - ETA: 128s - loss: 0.0126 - acc: 0.9961

60000/60000 [==============================] - 164s - loss: 0.0159 - acc: 0.9951 - val_loss: 0.0394 - val_acc: 0.9873
Epoch 10/10
10240/60000 [====>.........................] - ETA: 129s - loss: 0.0117 - acc: 0.9962

60000/60000 [==============================] - 163s - loss: 0.0141 - acc: 0.9952 - val_loss: 0.0476 - val_acc: 0.9867


In [23]:
simpleCNN.optimizer.lr=0.0001
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 164s - loss: 0.0128 - acc: 0.9957 - val_loss: 0.0421 - val_acc: 0.9876


## Data Augmentation

同样是利用ImageDataGenerator, 传入Data Augmentation的参数. 参数规定了随机对图片进行旋转、缩放等操作等角度、幅度范围

In [0]:
batch_size = 64
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, shear_range=0.5)
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

In [10]:
simpleCNN = getSimpleCNN()
simpleCNN.optimizer.lr=0.001
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/3
18176/60000 [========>.....................] - ETA: 16s - loss: 0.5300 - acc: 0.8278

60000/60000 [==============================] - 22s - loss: 0.2638 - acc: 0.9152 - val_loss: 0.0901 - val_acc: 0.9690
Epoch 2/3
60000/60000 [==============================] - 21s - loss: 0.0870 - acc: 0.9730 - val_loss: 0.0621 - val_acc: 0.9821
Epoch 3/3
 7744/60000 [==>...........................] - ETA: 16s - loss: 0.0773 - acc: 0.9765

60000/60000 [==============================] - 20s - loss: 0.0685 - acc: 0.9786 - val_loss: 0.0522 - val_acc: 0.9845


In [11]:
simpleCNN.optimizer.lr=0.0001
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 20s - loss: 0.0598 - acc: 0.9814 - val_loss: 0.0526 - val_acc: 0.9838
Epoch 2/3
60000/60000 [==============================] - 20s - loss: 0.0558 - acc: 0.9831 - val_loss: 0.0433 - val_acc: 0.9855
Epoch 3/3
  896/60000 [..............................] - ETA: 16s - loss: 0.0460 - acc: 0.9855

60000/60000 [==============================] - 20s - loss: 0.0491 - acc: 0.9852 - val_loss: 0.0495 - val_acc: 0.9848


In [12]:
simpleCNN.optimizer.lr=0.00001
simpleCNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.0467 - acc: 0.9857 - val_loss: 0.0349 - val_acc: 0.9893


## Batch Normalization

注意, 对Batch Normalization要设定其axis为色彩通道所对应的axis, 默认为-1也是可以的

In [0]:
bn_model = Sequential([
    Lambda(norm_input, input_shape=(28, 28, 1)),
    Convolution2D(32, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(32, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(),
    Convolution2D(64, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(64, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(axis=1),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
bn_model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
bn_model.optimizer.lr = 0.001
bn_model.fit_generator(batches, trn_label.shape[0], validation_data=test_batches, nb_val_samples=test_label.shape[0], nb_epoch=3)

Epoch 1/3
60000/60000 [==============================] - 30s - loss: 0.0603 - acc: 0.9815 - val_loss: 0.0535 - val_acc: 0.9843
Epoch 2/3
31776/60000 [==============>...............] - ETA: 13s - loss: 0.0569 - acc: 0.9826

60000/60000 [==============================] - 30s - loss: 0.0545 - acc: 0.9835 - val_loss: 0.0472 - val_acc: 0.9855
Epoch 3/3
49312/60000 [=======================>......] - ETA: 5s - loss: 0.0480 - acc: 0.9852

60000/60000 [==============================] - 30s - loss: 0.0472 - acc: 0.9854 - val_loss: 0.0426 - val_acc: 0.9856


In [18]:
bn_model.optimizer.lr = 0.0001
bn_model.fit_generator(batches, trn_label.shape[0], validation_data=test_batches, nb_val_samples=test_label.shape[0], nb_epoch=3)

Epoch 1/3
60000/60000 [==============================] - 30s - loss: 0.0459 - acc: 0.9861 - val_loss: 0.0668 - val_acc: 0.9799
Epoch 2/3
32032/60000 [===============>..............] - ETA: 13s - loss: 0.0433 - acc: 0.9868

60000/60000 [==============================] - 30s - loss: 0.0416 - acc: 0.9871 - val_loss: 0.0450 - val_acc: 0.9862
Epoch 3/3
49568/60000 [=======================>......] - ETA: 4s - loss: 0.0379 - acc: 0.9883

60000/60000 [==============================] - 30s - loss: 0.0378 - acc: 0.9885 - val_loss: 0.0378 - val_acc: 0.9881


In [19]:
bn_model.optimizer.lr = 0.00001
bn_model.fit_generator(batches, trn_label.shape[0], validation_data=test_batches, nb_val_samples=test_label.shape[0], nb_epoch=1)

Epoch 1/1
60000/60000 [==============================] - 30s - loss: 0.0385 - acc: 0.9882 - val_loss: 0.0359 - val_acc: 0.9893


# Drop Out

In [0]:
bndo_model = Sequential([
    Lambda(norm_input, input_shape=(28, 28, 1)),
    Convolution2D(32, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(32, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(),
    Convolution2D(64, 3, 3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(64, 3, 3, activation="relu"),
    MaxPooling2D(),
    BatchNormalization(axis=1),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
bndo_model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
bndo_model.optimizer.lr=0.001
bndo_model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 39s - loss: 0.1218 - acc: 0.9638 - val_loss: 0.0339 - val_acc: 0.9896
Epoch 2/3
 3296/60000 [>.............................] - ETA: 34s - loss: 0.0505 - acc: 0.9839

60000/60000 [==============================] - 38s - loss: 0.0535 - acc: 0.9842 - val_loss: 0.0317 - val_acc: 0.9910
Epoch 3/3
22752/60000 [==========>...................] - ETA: 22s - loss: 0.0373 - acc: 0.9879

60000/60000 [==============================] - 38s - loss: 0.0400 - acc: 0.9876 - val_loss: 0.0321 - val_acc: 0.9888


In [22]:
bndo_model.optimizer.lr=0.0001
bndo_model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 38s - loss: 0.0347 - acc: 0.9893 - val_loss: 0.0293 - val_acc: 0.9906
Epoch 2/3
 3392/60000 [>.............................] - ETA: 34s - loss: 0.0312 - acc: 0.9903

60000/60000 [==============================] - 38s - loss: 0.0287 - acc: 0.9913 - val_loss: 0.0226 - val_acc: 0.9938
Epoch 3/3
22784/60000 [==========>...................] - ETA: 22s - loss: 0.0229 - acc: 0.9925

60000/60000 [==============================] - 38s - loss: 0.0238 - acc: 0.9924 - val_loss: 0.0225 - val_acc: 0.9936


In [23]:
bndo_model.optimizer.lr=0.00001
bndo_model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch5=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 38s - loss: 0.0213 - acc: 0.9934 - val_loss: 0.0286 - val_acc: 0.9918
